<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>SECTION 1 - CREATING DATASET</strong></p>

<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>1. LIBRARIES</strong></p>

In [1]:
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from seleniumbase import Driver
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os
import glob
import sys
from config import project_root, final_file_path # custom config module
import winshell
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from IPython.display import display

<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>1. CREATE PROJECT ROOT</strong></p>

In [2]:
# Define and Create Project Root
if project_root == '{your_project_folder}':
    path_desktop = winshell.desktop()
    project_root = os.path.join(path_desktop, 'Project Eksi Sozluk')
    
    # Create folder if it doesn't exist
    if not os.path.exists(project_root):
        os.makedirs(project_root)
        print("Folder created:", project_root)
    else:
        print("Folder already exists:", project_root)
print('Your project folder: ', project_root)

Your project folder:  C:\Users\soner.kar\Desktop\My Projects\SözlükLens


<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>3. WEB SCRAPING-SELENIUM</strong></p>

In [3]:
# Start Selenium WebDrive
driver = Driver(uc=True)
driver.maximize_window()

# Get desktop path
path_desktop = winshell.desktop()

# Get DataFrame
df_path = fr"{project_root}\dataset.xlsx"
if not os.path.exists(df_path):
    columns = ['img_index', 'img_url', 'entry_favorites', 'entry_date', 'entry_edit_date', 
               'entry_id', 'entry_content', 'entry_author_name', 'entry_author_id']
    df = pd.DataFrame(columns=columns)
    df.to_excel(df_path, index=False)
    print(f"'{df_path}' was created.")
else:
    df = pd.read_excel(df_path)

In [4]:
# login.env
load_dotenv(dotenv_path=fr"{project_root}\env_login.env")
myEmail = os.getenv('MY_EKSISOZLUK_EMAIL')
myPassword = os.getenv('MY_EKSISOZLUK_PASSWORD')

# Go to Ekşi Sözlük login page
url_login = "https://eksisozluk.com/giris"
driver.uc_open_with_reconnect(url_login, 10)
sleep(0.5)
driver.uc_gui_click_captcha()
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "input")))
sleep(1)
login_username = driver.find_element(By.ID, 'username')
login_password = driver.find_element(By.ID, 'password')
login_username.send_keys(myEmail)
login_password.send_keys(myPassword)
sleep(1)
login_button = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/section/div/form/fieldset/div[5]/button")
login_button.click()
sleep(1)

# Loop variables
list_ok_images = list()
list_not_ok_images = list()
number_of_pages_to_visit = 1500 # default
page_number = 0
img_index = 0
img_index_top_of_page = 0
img_url_base = "https://cdn.eksisozluk.com/"
link_url_base = "soz.lk"

# Get the total number of pages to visit
url_main = f"https://eksisozluk.com/sozluk-yazarlarinin-cektigi-fotograflar--5297538"
driver.get(url_main)
total_page_number_element = driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[2]/section/div[1]/div[1]/div[2]/a[1]")
number_of_pages_to_visit = int(total_page_number_element.text)
print(f'There are {number_of_pages_to_visit} pages to scrape data.')

There are 1758 pages to scrape data.


In [5]:
# Loop-1) Pages
###number_of_pages_to_visit = 4
for i in range(number_of_pages_to_visit):
    # Check page number
    page_number += 1
    img_index_except_current_page = img_index

    # Go to the relevant topic page
    sleep(1)
    url = f"https://eksisozluk.com/sozluk-yazarlarinin-cektigi-fotograflar--5297538?p={page_number}"
    driver.get(url)
    
    # Wait for the page to load
    sleep(2)
    
    # Find 'Görsel' (Image) links
    image_links = driver.find_elements(By.XPATH, "//a[contains(@href, 'https://soz.lk/i/') and contains(@class, 'url')]")

    # Print results every 100 pages
    if page_number % 100 == 0:
        print(f'Result on page: {page_number}')
        print('Length of Downloaded Images:     ', len(list_ok_images))
        print('Length of Not Downloaded Images: ', len(list_not_ok_images))

    # Loop-2) Entries
    for index, link in enumerate(image_links):
        try:
            link_url = link.get_attribute("href")

            # Find entry
            entry_element = link.find_element(By.XPATH, "./ancestor::li")
            # Get entry id
            entry_id = entry_element.get_attribute("data-id")
            
            # Get date
            entry_date_time_element = entry_element.find_element(By.CSS_SELECTOR, ".entry-date.permalink")
            entry_date = entry_date_time_element.text
            if len(entry_date) == 16:
                entry_edit_date = ''
            elif len(entry_date) == 24:
                entry_edit_date = ''
                entry_date = entry_date[:16]
            elif len(entry_date) == 35:
                entry_edit_date = entry_date[19:]
                entry_date = entry_date[:16]

            # Get favorites count
            entry_favorites_element = entry_element.find_element(By.CSS_SELECTOR, "a.favorite-count.toggles")
            entry_favorites = entry_favorites_element.text
            if entry_favorites == '':
                entry_favorites = '0'

            # Get author name
            entry_author_name = entry_element.get_attribute("data-author")

            # Get author id
            entry_author_id = entry_element.get_attribute("data-author-id")

            # Get entry content
            entry_content_element = entry_element.find_element(By.CSS_SELECTOR, "#entry-item > div.content")
            entry_content = entry_content_element.text
            
            # Generate image download url
            if link_url_base in link_url:
                # Generate image download url
                url_id = link_url.split('/')[4]
                if entry_date != entry_edit_date:
                    img_url_date = entry_date
                else:
                    img_url_date = entry_edit_date
                date_object = datetime.strptime(img_url_date, "%d.%m.%Y %H:%M")
                entry_day = str(date_object.day)
                entry_month = str(date_object.month)
                entry_year = str(date_object.year)
                img_url = img_url_base + entry_year + '/' + entry_month + '/' + entry_day + '/' + url_id[0] + '/' + url_id + '.jpg'

                if img_url:
                        img_index += 1
                        list_ok_images.append(img_index)
                        df.loc[len(df)] = [img_index, img_url, entry_favorites, entry_date, entry_edit_date, entry_id, entry_content, entry_author_name, entry_author_id]
                        if img_index % 5000 == 0: 
                            df.to_excel(f"checkpoint_dataset_{img_index}.xlsx", index=False)
                else:                         
                    print(f"There is no img_url for entry_id: {entry_id} and img_index: {img_index}")

                if img_index % 2500 == 0:
                    print(f'--Current img_index: {img_index}')
            else:
                print(f"{link_url_base} is not in for entry_id: {entry_id} and img_index: {img_index}")
    
        except Exception as e:
            print(f"Error!: {e}")
    
    number_of_images_previous_page =  img_index - img_index_except_current_page
    print(f"-Page Number: {page_number} | Number of images: {number_of_images_previous_page}")

-Page Number: 1 | Number of images: 9
-Page Number: 2 | Number of images: 0
-Page Number: 3 | Number of images: 0
-Page Number: 4 | Number of images: 28


<a id = "1"></a><p style = "font-size : 40px; color :#E7E8D1 ; font-family : 'Comic Sans MS'; text-align : center; background-color : #B85042; border-radius: 5px 5px;"><strong>2. RESULT</strong></p>

# Print result

In [6]:
print('Final Result')
print('Length of Ok Images:     ', len(list_ok_images))
print('Length of Not Ok Images: ', len(list_not_ok_images))

display(df.head())

Final Result
Length of Ok Images:      37
Length of Not Ok Images:  0


,img_index,img_url,entry_favorites,entry_date,entry_edit_date,entry_id,entry_content,entry_author_name,entry_author_id
0,1,https://cdn.eksisozluk.com/2014/9/2/y/ydqeqxbz...,102,02.09.2014 14:26,26.03.2021 00:00,45405875,pek aram yoktur fotoğraf ile ama şunları sever...,ciodeth,492579
1,2,https://cdn.eksisozluk.com/2014/9/2/j/jukc5bb4...,102,02.09.2014 14:26,26.03.2021 00:00,45405875,pek aram yoktur fotoğraf ile ama şunları sever...,ciodeth,492579
2,3,https://cdn.eksisozluk.com/2014/9/2/g/grh1enp8...,102,02.09.2014 14:26,26.03.2021 00:00,45405875,pek aram yoktur fotoğraf ile ama şunları sever...,ciodeth,492579
3,4,https://cdn.eksisozluk.com/2014/9/2/n/nqq1chs9...,102,02.09.2014 14:26,26.03.2021 00:00,45405875,pek aram yoktur fotoğraf ile ama şunları sever...,ciodeth,492579
4,5,https://cdn.eksisozluk.com/2014/9/2/x/xw5gxvgo...,102,02.09.2014 14:26,26.03.2021 00:00,45405875,pek aram yoktur fotoğraf ile ama şunları sever...,ciodeth,492579


# Save dataset, delete checkpoints and update config.py file

In [7]:
# File name and path
# The following variables 'img_index' and 'project_root' come from the custom config.py file
final_file_name = f"1_dataset_{img_index}.xlsx"
final_file_path = os.path.join(project_root, final_file_name)

# Create the Excel file
df.to_excel(final_file_path, index=False)

# Check if the file exists
if os.path.exists(final_file_path):
    print(f'1) Dataset xlsx file was created: "{final_file_path}"')
    print(f'\n2) Checkpoint files was deleted.')
    
    # Delete checkpoint files
    files_to_delete = glob.glob(os.path.join(final_file_path, "checkpoint_dataset_*.xlsx"))
    os.remove("dataset.xlsx")
    
    for file_path in files_to_delete:
        try:
            os.remove(file_path)
            print(f'Deleted: "{file_path}"')
        except Exception as e:
            print(f'Error occurred: "{file_path}" - {e}')
else:
    print("Dataset xlsx file could not be created.")

# Update config.py 
# read file
with open("config.py", "r", encoding="utf-8") as f:
    lines = f.readlines()

# write file
with open("config.py", "w", encoding="utf-8") as f:
    for line in lines:
        if line.strip().startswith("img_index"):
            f.write(f"img_index = {img_index}\n")
            print(f'\n3) img_index variable was updated to {img_index} in config.py')
        else:
            f.write(line)

1) Dataset xlsx file was created: "C:\Users\soner.kar\Desktop\My Projects\SözlükLens\1_dataset_37.xlsx"

2) Checkpoint files was deleted.

3) img_index variable was updated to 37 in config.py
